In [1]:
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import csv
# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
#Keras
import keras
from keras import models
from keras import layers


Using TensorFlow backend.


In [10]:
# generating a dataset
header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate tempo'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' label'
header = header.split()


In [11]:
file = open('extra_data.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)

In [12]:
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
#genres = 'classical hiphop'.split()
for g in genres:
    for filename in os.listdir(f'../genres/{g}'):
        songname = f'../genres/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=None)
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        rmse = librosa.feature.rmse(y=y)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        tempo = librosa.feature.tempogram(y,sr)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)} {np.mean(tempo)}'    
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        to_append += f' {g}'
        file = open('extra_data.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())               


/home/anil/anaconda3/lib/python3.6/site-packages/scipy/fftpack/basic.py:159: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  z[index] = x


In [30]:
# import modules
import madmom 
import IPython.display as ipd 

# approach 2 - dbn tracker
proc = madmom.features.beats.DBNBeatTrackingProcessor(fps=100)
act = madmom.features.beats.RNNBeatProcessor()(songname)

beat_times = proc(act)

clicks = librosa.clicks(beat_times, sr=sr, length=len(y))
ipd.Audio(y + clicks, rate=sr)

In [33]:
import hdf5_getters
h5 = hdf5_getters.open_h5_file_read("/home/anil/Machine_learning/audio_processing/millionsongsubset_full/MillionSongSubset/data/B/B/B/TRBBBFO128F931535D.h5")
duration = hdf5_getters.get_duration(h5)
h5.close()

ModuleNotFoundError: No module named 'hdf5_getters'

In [34]:
!pip install hdf5_getters

  ERROR: Could not find a version that satisfies the requirement hdf5_getters (from versions: none)
ERROR: No matching distribution found for hdf5_getters
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
import pandas as pd

In [5]:
# reading dataset from csv

data = pd.read_csv('../csv_files/raj_corr_data.csv')
data.head()

# Dropping unneccesary columns
'''data = data.drop(['filename'],axis=1)
data.head()'''


"data = data.drop(['filename'],axis=1)\ndata.head()"

In [8]:
data.head()

,Unnamed: 0,spectral_bandwidth,zero_crossing_rate,rolloff,mfcc2,mfcc7,mfcc8,mfcc11,label
0,0,1972.202261,0.058236,3470.404015,115.346103,-9.058338,20.055258,-5.689818,blues
1,1,2310.305516,0.075345,4849.495676,93.629480,-9.776674,17.639806,-22.199284,blues
2,2,1772.854380,0.057147,2754.525010,139.977099,-11.213042,27.246107,-1.197161,blues
3,3,2290.401739,0.048052,4551.155813,89.791041,-5.403830,26.500496,-20.552354,blues
4,4,2356.855937,0.097657,5108.319111,90.701812,-13.171819,20.535274,-21.433580,blues


In [71]:
genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)
print(y)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 

In [72]:
# normalizing
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))

# spliting of dataset into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


In [73]:
# creating a model
model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))

model.add(layers.Dense(128, activation='relu'))

model.add(layers.Dense(64, activation='relu'))

model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
              
history = model.fit(X_train,
                    y_train,
                    epochs=500,
                    batch_size=128)

Epoch 1/500
792/792 [==============================] - 1s 1ms/step - loss: 2.2093 - acc: 0.2273
Epoch 2/500
792/792 [==============================] - 0s 45us/step - loss: 1.9495 - acc: 0.3460
Epoch 3/500
792/792 [==============================] - 0s 42us/step - loss: 1.6887 - acc: 0.4242
Epoch 4/500
792/792 [==============================] - 0s 41us/step - loss: 1.4263 - acc: 0.5152
Epoch 5/500
792/792 [==============================] - 0s 45us/step - loss: 1.2028 - acc: 0.5960
Epoch 6/500
792/792 [==============================] - 0s 40us/step - loss: 1.0206 - acc: 0.6616
Epoch 7/500
792/792 [==============================] - 0s 44us/step - loss: 0.8871 - acc: 0.7134
Epoch 8/500
792/792 [==============================] - 0s 42us/step - loss: 0.7930 - acc: 0.7298
Epoch 9/500
792/792 [==============================] - 0s 43us/step - loss: 0.7120 - acc: 0.7500
Epoch 10/500
792/792 [==============================] - 0s 44us/step - loss: 0.6554 - acc: 0.7715
Epoch 11/500
792/792 [========

792/792 [==============================] - 0s 47us/step - loss: 0.0144 - acc: 0.9987
Epoch 168/500
792/792 [==============================] - 0s 45us/step - loss: 0.0145 - acc: 0.9975
Epoch 169/500
792/792 [==============================] - 0s 40us/step - loss: 0.0150 - acc: 0.9975
Epoch 170/500
792/792 [==============================] - 0s 39us/step - loss: 0.0150 - acc: 0.9987
Epoch 171/500
792/792 [==============================] - 0s 44us/step - loss: 0.0136 - acc: 1.0000
Epoch 172/500
792/792 [==============================] - 0s 44us/step - loss: 0.0129 - acc: 1.0000
Epoch 173/500
792/792 [==============================] - 0s 44us/step - loss: 0.0114 - acc: 1.0000
Epoch 174/500
792/792 [==============================] - 0s 43us/step - loss: 0.0106 - acc: 1.0000
Epoch 175/500
792/792 [==============================] - 0s 39us/step - loss: 0.0101 - acc: 1.0000
Epoch 176/500
792/792 [==============================] - 0s 44us/step - loss: 0.0110 - acc: 1.0000
Epoch 177/500
792/792 [=

792/792 [==============================] - 0s 49us/step - loss: 0.0015 - acc: 1.0000
Epoch 332/500
792/792 [==============================] - 0s 47us/step - loss: 0.0014 - acc: 1.0000
Epoch 333/500
792/792 [==============================] - 0s 40us/step - loss: 0.0015 - acc: 1.0000
Epoch 334/500
792/792 [==============================] - 0s 43us/step - loss: 0.0014 - acc: 1.0000
Epoch 335/500
792/792 [==============================] - 0s 41us/step - loss: 0.0014 - acc: 1.0000
Epoch 336/500
792/792 [==============================] - 0s 39us/step - loss: 0.0014 - acc: 1.0000
Epoch 337/500
792/792 [==============================] - 0s 44us/step - loss: 0.0014 - acc: 1.0000
Epoch 338/500
792/792 [==============================] - 0s 38us/step - loss: 0.0014 - acc: 1.0000
Epoch 339/500
792/792 [==============================] - 0s 40us/step - loss: 0.0014 - acc: 1.0000
Epoch 340/500
792/792 [==============================] - 0s 39us/step - loss: 0.0015 - acc: 1.0000
Epoch 341/500
792/792 [=

792/792 [==============================] - 0s 43us/step - loss: 4.3985e-04 - acc: 1.0000
Epoch 492/500
792/792 [==============================] - 0s 43us/step - loss: 4.4302e-04 - acc: 1.0000
Epoch 493/500
792/792 [==============================] - 0s 38us/step - loss: 4.3629e-04 - acc: 1.0000
Epoch 494/500
792/792 [==============================] - 0s 40us/step - loss: 4.3251e-04 - acc: 1.0000
Epoch 495/500
792/792 [==============================] - 0s 38us/step - loss: 4.3744e-04 - acc: 1.0000
Epoch 496/500
792/792 [==============================] - 0s 43us/step - loss: 4.2002e-04 - acc: 1.0000
Epoch 497/500
792/792 [==============================] - 0s 43us/step - loss: 4.3624e-04 - acc: 1.0000
Epoch 498/500
792/792 [==============================] - 0s 47us/step - loss: 4.3123e-04 - acc: 1.0000
Epoch 499/500
792/792 [==============================] - 0s 46us/step - loss: 4.1645e-04 - acc: 1.0000
Epoch 500/500
792/792 [==============================] - 0s 40us/step - loss: 4.2578e-0

In [75]:
# calculate accuracy
test_loss, test_acc = model.evaluate(X_test,y_test)
print('test_acc: ',test_acc)

198/198 [==============================] - 0s 62us/step
test_acc:  0.9494949494949495


In [8]:

for filename in os.listdir(f'./test_audio/'):
    songname = f'./test_audio/{filename}'
    y, sr = librosa.load(songname, mono=True, duration=30)
    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
    rmse = librosa.feature.rmse(y=y)
    spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
    spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    zcr = librosa.feature.zero_crossing_rate(y)
    mfcc = librosa.feature.mfcc(y=y, sr=sr)
    to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
    for e in mfcc:
        to_append += f' {np.mean(e)}'
    to_append += f' {g}'
    file = open('test_data.csv', 'a', newline='')
    with file:
        writer = csv.writer(file)
        writer.writerow(to_append.split())


NameError: name 'g' is not defined

In [7]:
def predict_class(prediction):
    if prediction == 0:
        return "blues"
    elif prediction == 1:
        return "classical"
    elif prediction == 2:
        return "country"
    elif prediction == 3:
        return "disco"
    elif prediction == 4:
        return "hiphop"
    elif prediction == 5:
        return "jazz"
    elif prediction == 6:
        return "metal"
    elif prediction == 7:
        return "pop"
    elif prediction == 8:
        return "raggae"
    else:
        return "rock"
    

In [76]:
# reading dataset from csv

test_data = pd.read_csv('test_corr_data.csv')
test_data.head()

# Dropping unneccesary columns
'''test_data = test_data.drop(['filename'],axis=1)
test_data = test_data.drop(['label'],axis=1)
test_data.head()'''

"test_data = test_data.drop(['filename'],axis=1)\ntest_data = test_data.drop(['label'],axis=1)\ntest_data.head()"

In [77]:
correct = 0
wrong = 0
#predictions = model.predict(test_data)
predictions = model.predict(X_test)
for index in range(len(predictions)): 
    prediction = np.argmax(predictions[index])
    audio_name = predict_class(prediction)
    print(predict_class(y_test[index]))
    print(audio_name)
    if predict_class(y_test[index]) == audio_name:
        correct = correct+1
    else:
        wrong = wrong+1
        
    print("***************************************************")
print("correct",correct)
print("wrong",wrong)

rock
rock
***************************************************
raggae
pop
***************************************************
raggae
raggae
***************************************************
raggae
raggae
***************************************************
metal
metal
***************************************************
raggae
raggae
***************************************************
disco
disco
***************************************************
pop
pop
***************************************************
metal
metal
***************************************************
jazz
jazz
***************************************************
jazz
jazz
***************************************************
country
country
***************************************************
blues
blues
***************************************************
blues
blues
***************************************************
raggae
raggae
***************************************************
metal
metal
***************************

In [2]:
fsd_data = pd.read_csv("/home/anil/Downloads/train.csv")

In [3]:
fsd_data

,fname,label,manually_verified
0,00044347.wav,Hi-hat,0
1,001ca53d.wav,Saxophone,1
2,002d256b.wav,Trumpet,0
3,0033e230.wav,Glockenspiel,1
4,00353774.wav,Cello,1
5,003b91e8.wav,Cello,0
6,003da8e5.wav,Knock,1
7,0048fd00.wav,Gunshot_or_gunfire,1
8,004ad66f.wav,Clarinet,0
9,0063ab88.wav,Computer_keyboard,0


In [8]:
fsd_data["label"].unique()

array(['Hi-hat', 'Saxophone', 'Trumpet', 'Glockenspiel', 'Cello', 'Knock',
       'Gunshot_or_gunfire', 'Clarinet', 'Computer_keyboard',
       'Keys_jangling', 'Snare_drum', 'Writing', 'Laughter', 'Tearing',
       'Fart', 'Oboe', 'Flute', 'Cough', 'Telephone', 'Bark', 'Chime',
       'Bass_drum', 'Bus', 'Squeak', 'Scissors', 'Harmonica', 'Gong',
       'Microwave_oven', 'Burping_or_eructation', 'Double_bass',
       'Shatter', 'Fireworks', 'Tambourine', 'Cowbell', 'Electric_piano',
       'Meow', 'Drawer_open_or_close', 'Applause', 'Acoustic_guitar',
       'Violin_or_fiddle', 'Finger_snapping'], dtype=object)

In [ ]:
fout=open("combined_csv_out.csv","a")
# first file:
for line in open("../fma_metadata/features.csv"):
    fout.write(line)

track = pd.read_csv('../fma_metadata/tracks.csv')
fout.write(track["track.7"])

fout.close()


In [13]:
import pandas as pd
fields = ['track.7']

df = pd.read_csv('../fma_metadata/tracks.csv', skipinitialspace=True, usecols=fields)
# See the keys
print(df.keys())
# See content in 'star_name'
print(df.star_name)

Index(['track.7'], dtype='object')


AttributeError: 'DataFrame' object has no attribute 'star_name'

In [14]:
df = pd.read_csv('../fma_metadata/tracks.csv')
df["track.7"]

/home/anil/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (0,1,5,6,8,12,18,20,21,22,24,33,34,38,39,44,47,49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


0            genre_top
1                  NaN
2              Hip-Hop
3              Hip-Hop
4              Hip-Hop
5                  Pop
6                  NaN
7                  NaN
8                  NaN
9                  NaN
10                 NaN
11             Hip-Hop
12                Rock
13                Rock
14        Experimental
15        Experimental
16                Folk
17                Folk
18                Folk
19                Folk
20                Jazz
21                Jazz
22                Jazz
23                Jazz
24        Experimental
25        Experimental
26        Experimental
27                Rock
28                Rock
29                Rock
              ...     
106546    Instrumental
106547    Instrumental
106548    Instrumental
106549    Instrumental
106550    Instrumental
106551    Instrumental
106552    Instrumental
106553    Instrumental
106554            Folk
106555            Folk
106556            Folk
106557            Folk
106558     

In [3]:
import pandas as pd 

In [4]:
df1 = pd.read_csv('../fma_metadata/features.csv')

/home/anil/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249

In [5]:
df2 = pd.read_csv('../fma_metadata/tracks.csv')

/home/anil/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (0,1,5,6,8,12,18,20,21,22,24,33,34,38,39,44,47,49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
print(df1.columns)
print(df2.columns)

Index(['feature', 'chroma_cens', 'chroma_cens.1', 'chroma_cens.2',
       'chroma_cens.3', 'chroma_cens.4', 'chroma_cens.5', 'chroma_cens.6',
       'chroma_cens.7', 'chroma_cens.8',
       ...
       'tonnetz.39', 'tonnetz.40', 'tonnetz.41', 'zcr', 'zcr.1', 'zcr.2',
       'zcr.3', 'zcr.4', 'zcr.5', 'zcr.6'],
      dtype='object', length=519)
Index(['Unnamed: 0', 'album', 'album.1', 'album.2', 'album.3', 'album.4',
       'album.5', 'album.6', 'album.7', 'album.8', 'album.9', 'album.10',
       'album.11', 'album.12', 'artist', 'artist.1', 'artist.2', 'artist.3',
       'artist.4', 'artist.5', 'artist.6', 'artist.7', 'artist.8', 'artist.9',
       'artist.10', 'artist.11', 'artist.12', 'artist.13', 'artist.14',
       'artist.15', 'artist.16', 'set', 'set.1', 'track', 'track.1', 'track.2',
       'track.3', 'track.4', 'track.5', 'track.6', 'track.7', 'track.8',
       'track.9', 'track.10', 'track.11', 'track.12', 'track.13', 'track.14',
       'track.15', 'track.16', 'track.17', 'tra

In [7]:
df1.feature.isnull().any()

False

In [8]:
df2["Unnamed: 0"].isnull().sum()

1

In [13]:
len(df1.feature)

106577

In [14]:
len(df2["Unnamed: 0"])

106576

In [22]:
d1=df1[0:10]

In [23]:
d1

,feature,chroma_cens,chroma_cens.1,chroma_cens.2,chroma_cens.3,chroma_cens.4,chroma_cens.5,chroma_cens.6,chroma_cens.7,chroma_cens.8,...,tonnetz.39,tonnetz.40,tonnetz.41,zcr,zcr.1,zcr.2,zcr.3,zcr.4,zcr.5,zcr.6
0,statistics,kurtosis,kurtosis,kurtosis,kurtosis,kurtosis,kurtosis,kurtosis,kurtosis,kurtosis,...,std,std,std,kurtosis,max,mean,median,min,skew,std
1,number,01,02,03,04,05,06,07,08,09,...,04,05,06,01,01,01,01,01,01,01
2,track_id,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,7.1806526184e+00,5.2303090096e+00,2.4932080507e-01,1.3476201296e+00,1.4824777842e+00,5.3137123585e-01,1.4815930128e+00,2.6914546490e+00,8.6686819792e-01,...,5.4125156254e-02,1.2225749902e-02,1.2110591866e-02,5.7588901520e+00,4.5947265625e-01,8.5629448295e-02,7.1289062500e-02,0.0000000000e+00,2.0898721218e+00,6.1448108405e-02
4,3,1.8889633417e+00,7.6053929329e-01,3.4529656172e-01,2.2952005863e+00,1.6540306807e+00,6.7592434585e-02,1.3668476343e+00,1.0540937185e+00,1.0810308903e-01,...,6.3831120729e-02,1.4211839065e-02,1.7740072682e-02,2.8246941566e+00,4.6630859375e-01,8.4578499198e-02,6.3964843750e-02,0.0000000000e+00,1.7167237997e+00,6.9330163300e-02
5,5,5.2756297588e-01,-7.7654317021e-02,-2.7961030602e-01,6.8588310480e-01,1.9375696182e+00,8.8083887100e-01,-9.2319184542e-01,-9.2723226547e-01,6.6661673784e-01,...,4.0730185807e-02,1.2690781616e-02,1.4759079553e-02,6.8084154129e+00,3.7500000000e-01,5.3114086390e-02,4.1503906250e-02,0.0000000000e+00,2.1933031082e+00,4.4860601425e-02
6,10,3.7022454739e+00,-2.9119303823e-01,2.1967420578e+00,-2.3444947600e-01,1.3673638105e+00,9.9841135740e-01,1.7706941366e+00,1.6045658588e+00,5.2121698856e-01,...,7.4357867241e-02,1.7951935530e-02,1.3921394013e-02,2.1434211731e+01,4.5214843750e-01,7.7514506876e-02,7.1777343750e-02,0.0000000000e+00,3.5423245430e+00,4.0800448507e-02
7,20,-1.9383698702e-01,-1.9852678478e-01,2.0154602826e-01,2.5855624676e-01,7.7520370483e-01,8.4794059396e-02,-2.8929358721e-01,-8.1641042233e-01,4.3850939721e-02,...,9.5002755523e-02,2.2492416203e-02,2.1355332807e-02,1.6669036865e+01,4.6972656250e-01,4.7224905342e-02,4.0039062500e-02,9.7656250000e-04,3.1898307800e+00,3.0992921442e-02
8,26,-6.9953453541e-01,-6.8415790796e-01,4.8824872822e-02,4.2658798397e-02,-8.1896692514e-01,-9.1712284088e-01,-9.0183424950e-01,-6.6844828427e-02,-2.9103723168e-01,...,1.0371652246e-01,2.5541320443e-02,2.3846302181e-02,4.1645809174e+01,2.5048828125e-01,1.8387714401e-02,1.5625000000e-02,0.0000000000e+00,4.6905956268e+00,1.4598459937e-02
9,30,-7.2148716450e-01,-8.4855991602e-01,8.9090377092e-01,8.8619679213e-02,-4.4551330805e-01,-1.2711701393e+00,-1.2401897907e+00,-1.3437650204e+00,-9.0560036898e-01,...,1.4169253409e-01,2.0426128060e-02,2.5417611003e-02,8.1665945053e+00,5.4687500000e-01,5.4416511208e-02,3.6132812500e-02,2.4414062500e-03,2.2447082996e+00,5.2673552185e-02


In [24]:
d2=df2[0:10]

In [25]:
d2

,Unnamed: 0,album,album.1,album.2,album.3,album.4,album.5,album.6,album.7,album.8,...,track.10,track.11,track.12,track.13,track.14,track.15,track.16,track.17,track.18,track.19
0,NaN,comments,date_created,date_released,engineer,favorites,id,information,listens,producer,...,information,interest,language_code,license,listens,lyricist,number,publisher,tags,title
1,track_id,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4,1,<p></p>,6073,NaN,...,NaN,4656,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,1293,NaN,3,NaN,[],Food
3,3,0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4,1,<p></p>,6073,NaN,...,NaN,1470,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,514,NaN,4,NaN,[],Electric Ave
4,5,0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4,1,<p></p>,6073,NaN,...,NaN,1933,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,1151,NaN,6,NaN,[],This World
5,10,0,2008-11-26 01:45:08,2008-02-06 00:00:00,NaN,4,6,NaN,47632,NaN,...,NaN,54881,en,Attribution-NonCommercial-NoDerivatives (aka M...,50135,NaN,1,NaN,[],Freeway
6,20,0,2008-11-26 01:45:05,2009-01-06 00:00:00,NaN,2,4,"<p> ""spiritual songs"" from Nicky Cook</p>",2710,NaN,...,NaN,978,en,Attribution-NonCommercial-NoDerivatives (aka M...,361,NaN,3,NaN,[],Spiritual Level
7,26,0,2008-11-26 01:45:05,2009-01-06 00:00:00,NaN,2,4,"<p> ""spiritual songs"" from Nicky Cook</p>",2710,NaN,...,NaN,1060,en,Attribution-NonCommercial-NoDerivatives (aka M...,193,NaN,4,NaN,[],Where is your Love?
8,30,0,2008-11-26 01:45:05,2009-01-06 00:00:00,NaN,2,4,"<p> ""spiritual songs"" from Nicky Cook</p>",2710,NaN,...,NaN,718,en,Attribution-NonCommercial-NoDerivatives (aka M...,612,NaN,5,NaN,[],Too Happy
9,46,0,2008-11-26 01:45:05,2009-01-06 00:00:00,NaN,2,4,"<p> ""spiritual songs"" from Nicky Cook</p>",2710,NaN,...,NaN,252,en,Attribution-NonCommercial-NoDerivatives (aka M...,171,NaN,8,NaN,[],Yosemite


In [28]:
d1.feature

0    statistics
1        number
2      track_id
3             2
4             3
5             5
6            10
7            20
8            26
9            30
Name: feature, dtype: object

In [30]:
d2["Unnamed: 0"]

0         NaN
1    track_id
2           2
3           3
4           5
5          10
6          20
7          26
8          30
9          46
Name: Unnamed: 0, dtype: object

In [29]:
print (pd.merge(df1, df2, left_on='feUnnamed: 0ature', right_on='Unnamed: 0', how='left')
         .drop(['Unnamed: 0'], axis=1))

KeyError: 'feUnnamed: 0ature'

In [44]:
left=pd.merge(df1, df2, left_on='feature', right_on='Unnamed: 0', how='left').drop(['Unnamed: 0'], axis=1)

In [42]:
df1.feature.count()
# feature           106577

106577

In [41]:
df2["Unnamed: 0"].count()
#Unnamed: 0    106575

106575

In [45]:
left.feature.count()

106577

In [46]:
left['track.7'].count()

40432

In [48]:
left['track.7'].isnull().sum()

66145

In [49]:
66145+40432

106577

In [50]:
df2['track.7'].isnull().sum()

56977

In [51]:
right=pd.merge(df1, df2, left_on='feature', right_on='Unnamed: 0', how='right')

In [55]:
len(right.columns)

572

In [56]:
right['track.7'].isnull().sum()

56977

In [58]:
right.feature.isnull().sum()

15362

In [ ]:
right=pd.merge(df1, df2, left_on='feature', right_on='Unnamed: 0', how='right')

In [1]:
df1

NameError: name 'df1' is not defined